In [1]:
import pandas as pd
import numpy as np



In [11]:
order_events = pd.read_csv("csv/order_events.csv")
picked_up = order_events[order_events['event_status'] == 'PICKED_UP']
delivered = order_events[order_events['event_status'] == 'DELIVERED']


In [16]:
order_events['event_timestamp'] = pd.to_datetime(order_events['event_timestamp'])


In [18]:
order_events_filtered = order_events[
    order_events['event_status'].isin(['PICKED_UP', 'DELIVERED'])
]


In [20]:
#Pivot Vertical Logs into Horizontal Form
delivery_times = (
    order_events_filtered
    .groupby('order_id')
    .agg(
        picked_up_time = (
            'event_timestamp',
            lambda x: x[order_events_filtered.loc[x.index, 'event_status'] == 'PICKED_UP'].min()
        ),
        delivered_time = (
            'event_timestamp',
            lambda x: x[order_events_filtered.loc[x.index, 'event_status'] == 'DELIVERED'].min()
        )
    )
    .reset_index()
)


In [22]:
#generating field actual delivery mins
delivery_times['actual_delivery_mins'] = (
    delivery_times['delivered_time'] - delivery_times['picked_up_time']
).dt.total_seconds() / 60


In [23]:
# Save delivery time metrics to CSV
delivery_times.to_csv(
    "delivery_times_actual_duration.csv",
    index=False
)
print("✅ delivery_times_actual_duration.csv saved successfully")
print("Rows:", delivery_times.shape[0])


✅ delivery_times_actual_duration.csv saved successfully
Rows: 50123
